In [1]:
from transformers import AutoModel
import torch
from huggingface_hub import get_full_repo_name
import os

model = AutoModel.from_pretrained("jinaai/jina-embeddings-v4", trust_remote_code=True, dtype = torch.float16)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModelForFeatureExtraction(
  (base_model): LoraModel(
    (model): JinaEmbeddingsV4Model(
      (model): Qwen2_5_VLModel(
        (visual): Qwen2_5_VisionTransformerPretrainedModel(
          (patch_embed): Qwen2_5_VisionPatchEmbed(
            (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
          )
          (rotary_pos_emb): Qwen2_5_VisionRotaryEmbedding()
          (blocks): ModuleList(
            (0-31): 32 x Qwen2_5_VLVisionBlock(
              (norm1): Qwen2RMSNorm((1280,), eps=1e-06)
              (norm2): Qwen2RMSNorm((1280,), eps=1e-06)
              (attn): Qwen2_5_VLVisionSdpaAttention(
                (qkv): Linear(in_features=1280, out_features=3840, bias=True)
                (proj): Linear(in_features=1280, out_features=1280, bias=True)
              )
              (mlp): Qwen2_5_VLMLP(
                (gate_proj): Linear(in_features=1280, out_features=3420, bias=True)
                (up_proj): Linear(in_features=1280, out_feat

In [2]:
#!/usr/bin/python
from configparser import ConfigParser

from sqlalchemy.sql.sqltypes import NULLTYPE


def config(filename='database.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db

In [3]:
#!/usr/bin/python
import psycopg2
from config import config

flag = None

def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model.to(device)

        # read connection parameters
        params = config()

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)

        # create a cursor
        cur = conn.cursor()

        return cur, conn
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

if __name__ == '__main__':
    cur, conn = connect()

Connecting to the PostgreSQL database...


In [4]:
import numpy as np
from transformers import AutoModel
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import psycopg2
from config import config


def read_column(cur):
    cur.execute('select id, name, description, price, image_url from Flower where vector IS NULL')
    if cur is None:
        print('No data found')
    return cur

read_column(cur)

def embedded_column(cur, conn):
    try:
        rows = cur.fetchall()
        print(f"🧩 Found {len(rows)} rows to embed...")
        for row in rows:
            flower_id, name, description, price, image = row

            text = text = f"{name}. {description or ''}. Price: ${price:.2f}"

            text_embedding = model.encode_text(
                texts = text,
                task = "retrieval",
                return_numpy = True,
            )
            text_embedding = text_embedding.squeeze().astype(np.float16).tolist()

            cur.execute('Update Flower Set vector = %s where id = %s', (text_embedding, flower_id))
    except(Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        conn.commit()
        conn.close()
        cur.close()
        print('Database connection closed.')
    return 0

def cosine_similarity(a, b):
    a = np.array(a)
    b = np.array(b)
    return np.dot(a, b.T) / (np.linalg.norm(a, axis=1)[:, None] * np.linalg.norm(b, axis=1))

embedded_column(cur, conn)

🧩 Found 0 rows to embed...
Database connection closed.


0

In [28]:
import ast
def retrieval_vector(query):
    try:
        cur, conn = connect()
        query_embedding = np.array(model.encode_text(
            texts = query,
            task = "retrieval",
            return_numpy = True,
        )).reshape(1, -1)

        cur.execute("select vector from Flower where vector IS NOT NULL")

        rows = cur.fetchall()

        flower_vectors = []
        for row in rows:
            vector = np.array(ast.literal_eval(row[0])).reshape(-1)
            flower_vectors.append(vector)

        flower_vectors = np.array(flower_vectors)

        top_5_index = np.array(np.argpartition(np.array(cosine_similarity(query_embedding, flower_vectors)).flatten(), -5)[-5:]) + 1

        result = []
        for index in top_5_index:
            cur.execute("select name, description, price, image_url from Flower where id = %s", (int(index),))
            result.append(cur.fetchone())

        for row in result:
            print(row[0], row[1], row[2], row[3])

    except(Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        conn.commit()
        conn.close()
        cur.close()
        print('Database connection closed.')
    return 0



retrieval_vector("Orchid")




Connecting to the PostgreSQL database...


Encoding texts...: 100%|██████████| 1/1 [00:01<00:00,  1.37s/it]

Tulip Bright and cheerful spring flower, available in many colors. 1.80 https://example.com/images/tulip.jpg
Lavender Fragrant purple herb flower, symbolizing calm and peace. 4.00 https://example.com/images/lavender.jpg
Peony Large, lush blooms available in pink, red, or white. 6.50 https://example.com/images/peony.jpg
Hydrangea Clustered flowers that change color depending on soil pH. 5.00 https://example.com/images/hydrangea.jpg
Orchid Elegant indoor plant with delicate petals, popular as gifts. 12.50 https://example.com/images/orchid.jpg
Database connection closed.


0